In [11]:
import pandas as pd
import feature_engineering as fe
import sklift.datasets.datasets as sdd
from class_learners import SLearner, TLearner, CorrSTLearner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from metric import extendedERUPT

<p><h1><b>Causal Machine Learning with Uplift Model</b></h1></p>
<p><h4><b>The objective:</b> find the intervention that maximises positive outcome  </h4> </p>
<p><h4><b>The data:</b> Hillstrom dataset</h4></p>

64,000 randomly chosen customers, who last purchased within 12 months, have been been subject to the following email intervention:
<ul>
    <li>1/3 were randomly chosen to receive an e-mail campaign featuring Men's merchandise.</li>
    <li>1/3 were randomly chosen to receive an e-mail campaign featuring Women's merchandise.</li>
    <li>1/3 were randomly chosen to not receive an e-mail campaign.</li>
</ul>
The the behaviour of the customers was tracked up to two weeks after the email campaign.

<i>I will use uplift models to determine the best email intervention for each customer: which email option (Men's merch, Women's merch, no email) suits best for which individual customer in order to maximise the number of visits of the online store.</i>

<h5><b>Columns with information of before the email intervention:</b></h5>
<ul>
    <li><b>Recency: </b>Months since last purchase.</li>
    <li><b>History_Segment:</b> Categorization of dollars spent in the past year.</li>
    <li><b>History:</b> Actual dollar value spent in the past year.</li>
    <li><b>Mens:</b> 1/0 indicator, 1 = customer purchased Mens merchandise in the past year.</li>
    <li><b>Womens:</b> 1/0 indicator, 1 = customer purchased Womens merchandise in the past year.</li>
    <li><b>Zip_Code:</b> Classifies zip code as Urban, Suburban, or Rural.</li>
    <li><b>Newbie:</b> 1/0 indicator, 1 = New customer in the past twelve months.</li>
    <li><b>Channel:</b> Describes the channels the customer purchased from in the past year.</li>    
</ul>
<h5><b>Intervention column:</b></h5>
<ul>
<li><b>Segment:</b></li>
<ul>
    <li>Mens E-Mail</li>
    <li>Womens E-Mail</li>
    <li>No E-Mail</li>
</ul>
</ul>

<h5><b>Columns tracking the activity of the customers up to two weeks after the email intervention:</b></h5>
<ul>
    <li><b>Visit:</b> 1/0 indicator, 1 = Customer visited website in the following two weeks.</li>
    <li><b>Conversion: </b>1/0 indicator, 1 = Customer purchased merchandise in the following two weeks.</li>
    <li><b>Spend:</b> Actual dollars spent in the following two weeks.</li>
</ul>
<br>
<p><h4><b>Causal Machine Learning:</b></h4></b>
<p>We would like to know how an individual customer will react when being subjected to a specific intervention. However, we can't perform multiple interventions on the same individual in order to see what would be the best option. The best approach is to use randomised controlled trial data (RCT): in these datasets, the interventions have been given randomly to a large enough number of individuals in order to deduce causal inference. </p>
<p>For example, there are two possible interventions: send an email or send no email. My objective is that I would like customers to visit my store. In my database, I look at customer A and wonder whether I should send an email. The RCT data should be a large enough pool of people so that at least two people, who are very similar to my customer A, had been subject to either intervention "email" or intervention "no email". My model (meta-learner) should give me the probabilities of success of each intervention on customer A, or - if looking at the uplift - the difference of these probabilities. This should give me enough insight to take an informed decision.</p>
<p>The hillstrom data set is a randomised controlled trial dataset: the interventions have been given randomly to a large enough number of customers in order to deduce causal inferences. </p>

In [12]:
hillstrom = sdd.fetch_hillstrom(target_col='visit', data_home=None, dest_subdir=None, download_if_missing=True, return_X_y_t=False)
hillstrom.keys()

dict_keys(['data', 'target', 'treatment', 'DESCR', 'feature_names', 'target_name', 'treatment_name'])

In [13]:
hillstrom_features = hillstrom['data']
hillstrom_features.head(2)

,recency,history_segment,history,mens,womens,zip_code,newbie,channel
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web


In [14]:
hillstrom_target = hillstrom['target']
hillstrom_treatment = hillstrom['treatment']
hillstrom_treatment.head(2)

0    Womens E-Mail
1        No E-Mail
Name: segment, dtype: object

<p><h3><b>Feature engineering</b></h3></p>
Change the following columns to datatype `category`:
<ul>
    <li>mens</li>
    <li>womens</li>
    <li>newbi</li>
</ul>
<p>Change the following columns to datatype `category` and create dummies:
<ul>
    <li>zip_code</li>
    <li>channel</li>
</ul></p>
<p>Mapping the values ("no email", "Women's email", and "Men's email") of the treatment to 0, 1, and 2.</p>
<p>Dropping the column `history_segment`.</p>

In [15]:
hillstrom_features = fe.feature_engineering(hillstrom_features)
hillstrom_features.head(2)

,recency,history,mens,womens,newbie,zip_code_Rural,zip_code_Surburban,zip_code_Urban,channel_Multichannel,channel_Phone,channel_Web
0,0.603876,-0.194501,1,0,0,0,1,0,0,1,0
1,0.033679,0.169807,1,1,1,1,0,0,0,0,1


In [16]:
hillstrom_treatment = fe.treatment_category(hillstrom_treatment)
hillstrom_treatment.head(2)

0    1
1    0
Name: segment, dtype: int64

<p><h3><b>S-Learner, T-Learner, and correlated ST-Learner  </b></h3></p>
Meta-learners are frameworks to estimate the Conditional Average Treatment Effect (CATE) or "Uplift" using any machine learning estimators. In this case, I will use Meta-learners to build a recommendation machine based on uplift models. I will use the S-, T-, and correlated T-learner as base models. As estimators I will use XGBoost, Random Forest, and Logistic Regression for each learner. Instead of computing the uplift of each email intervention, I will compare the probabilities of each intervention and opt for the treatment with the highest probability.<br>
The following explanations assume binary treatment options.
<p><h4><b>S-Learner:</b></h4></p>
<p>Contrary to the T-learner, the S-learner uses only one machine learning model (single base).
$$
\mu = E[Y|X=x, W=w]
$$
with $\mu$ being average outcome, $Y$ the outcome, $X$ the covariate, and $W$ the treatment effect.</p>
<p>The CATE is </p>
$$\tau(x) = \hat{\mu}(x,w = 1) - \hat{\mu}(x,w = 0).$$
<p><h4><b>T-Learner:</b></h4></p>
<p>The T-learner uses a model for each treatment option:
$$\mu_{0} = E[Y(0)|X=x],\quad \mu_{1} = E[Y(1)|X=x]$$.</p>
<p>The CATE is $\tilde{\tau} = \mu_{1}(x) - \mu_{0}(x)$.</p>
<p><h4><b>correlated ST-Learner:</b></h4></p>
<p>The correlated ST-learner is a boosted mixture of both the S- and the T-learner. The train and test data will be prepared according to the T- and S-learner, respectively.There are four models to be considered: the base model and the three booster models. The base model is an xgboost on the whole train data. The three booster models are models for $$\mu = E[Y(i)|X=x,W=i]$$ each boosted from the base model, where $i = 0,1$. The CATE is the same as for the S-learner. </p>
<hr>
<h5><b>S-Learner:</b></h5>


In [7]:
s = SLearner(hillstrom_features, hillstrom_treatment, hillstrom_target)
best_proba_logreg_S = s.get_proba(LogisticRegression,random_state = 42)
best_proba_randfor_S = s.get_proba(RandomForestClassifier,max_depth=2,min_samples_leaf=2, random_state=42)
best_proba_xgb_S = s.get_proba(xgb.XGBClassifier, seed=42, use_label_encoder=False, max_depth=1,eval_metric="logloss",)

<h5><b>T-Learner:</b></h5>

In [8]:
t = TLearner(hillstrom_features, hillstrom_treatment, hillstrom_target)
best_proba_logreg_T = t.get_proba(LogisticRegression, 'segment', random_state = 42)
best_proba_randfor_T = t.get_proba(RandomForestClassifier,'segment', max_depth=2,min_samples_leaf=2, random_state=42)
best_proba_xgb_T = t.get_proba(xgb.XGBClassifier, 'segment', seed = 42, use_label_encoder=False, max_depth=1,eval_metric="logloss")

<h5><b>Correlated ST-Learner:</b></h5>

In [9]:
ct = CorrSTLearner(hillstrom_features, hillstrom_treatment, hillstrom_target)
ct.prepare_data()
best_proba_xgb_cT = ct.get_proba(seed=42, use_label_encoder=False, max_depth=1,eval_metric="logloss")

In [17]:
ee = extendedERUPT(best_proba_logreg_S,s.test_treatment)
erupt_value_test, erupt_value_bench_2, erupt_bench_values = ee.get_ERUPT_with_benchmark()

ERUPT model: 2.0000, sum of y: 21270, number of rows: 10635
ERUPT benchmark: 2.0000, sum of y: 21270, number of rows: 10635
Difference of ERUPT(model) and ERUPT(benchmark): 0.0000
ERUPT benchmark: 2.0000, sum of y: 21270, number of rows: 10635
Difference of ERUPT(model) and ERUPT(benchmark_2): 0.0000


In [18]:
ee = extendedERUPT(best_proba_xgb_cT, ct.test_treatment)
erupt_value_test, erupt_value_bench_2, erupt_bench_values = ee.get_ERUPT_with_benchmark()

ERUPT model: 1.6402, sum of y: 17491, number of rows: 10664
ERUPT benchmark: 1.6405, sum of y: 17522, number of rows: 10681
Difference of ERUPT(model) and ERUPT(benchmark): -0.0003
ERUPT benchmark: 1.6412, sum of y: 17479, number of rows: 10650
Difference of ERUPT(model) and ERUPT(benchmark_2): -0.0010
